In [8]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector, RFE
from sklearn.linear_model import LinearRegression
from tqdm.keras import TqdmCallback

#Data Import
df = pd.read_csv('Bikes.csv', encoding= 'unicode_escape')
X, y = df.drop(['Bikes'], axis =1), df['Bikes']

def coeff_determination(y, yhat):
    """ Calculates the R2"""
    from keras import backend as K
    SS_res = K.sum(K.square(y-yhat))
    SS_tot = K.sum(K.square(y -K.mean(yhat)))
    return (1-SS_res/(SS_tot + K.epsilon()))

def r2_adjusted(x, r2):
    """Calculates the R2 Adjusted"""
    #no. instances, params
    m, n = x.shape
    #calculate r_df
    r_df = (m-1)/(m-n)
    return 1-(r_df*(1-r2))
              
def TwoL_NN(num_feat, act, epoch, learnrate):
    """ Creates the Sequential Model as dataset has one output, inputs are placed in the model
       then output layer. The optimizer used for the NN is Adam.A plot of MSE vs Epochs and 
       R2 vs Epochs is printed at the end. R2, R2 Bar and R2 CV are returned.
    """
    model = Sequential()
    model.add(Input(shape=(num_feat,)))         #Input Layer
    model.add(Dense(units=1))                   #Output Layer
    optimizer = Adam(learning_rate=learnrate)   #Adam used as Optimizer
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', coeff_determination])
    fit_model = model.fit(x=x_train, y=y_train, epochs=epoch, batch_size=32, validation_split=0.2, verbose=0, callbacks=[TqdmCallback()])
    plt.clf()
    plt.plot([i for i in range(epoch)], fit_model.history['mse'], label="mse")
    plt.title(f"{act}: {learnrate}")
    plt.legend()
    plt.show()
    plt.plot([i for i in range(epoch)], fit_model.history['coeff_determination'], label="R2")
    plt.legend()
    return (fit_model.history['coeff_determination'][-1], r2_score(y_test, model(x_test)), r2_adjusted(x_train,fit_model.history['coeff_determination'][-1]))

# 2 Layer Model

In [9]:
#Splitting training and testing sets (80/20)
x_train, y_train = X[-7006:].to_numpy(), (y[-7006:]).to_numpy()
x_test, y_test   = X[:-7006].to_numpy(), (y[:-7006]).to_numpy()

#Calling Neural Net Function for 3 different activation functions and 3 different learning rates.
sig1, sig_cv1, sig_bar1 = TwoL_NN(11, 'sigmoid', 200, 0.1)
sig2, sig_cv2, sig_bar2 = TwoL_NN(11, 'sigmoid', 200, 0.01)
sig3, sig_cv3, sig_bar3 = TwoL_NN(11, 'sigmoid', 200, 0.001)

relu1, relu_cv1, relu_bar1 = TwoL_NN(11, 'relu', 200, 0.1)
relu2, relu_cv2, relu_bar2 = TwoL_NN(11, 'relu', 200, 0.01)
relu3, relu_cv3, relu_bar3 = TwoL_NN(11, 'relu', 500,  0.001)

elu1, elu_cv1, elu_bar1 = TwoL_NN(11, 'elu', 200, 0.1)
elu2, elu_cv2, elu_bar2 = TwoL_NN(11, 'elu', 200, 0.01)
elu3, elu_cv3, elu_bar3 = TwoL_NN(11, 'elu', 500, 0.001)

#Output
R2s = pd.DataFrame({"Learning Rate" :[0.1,.01,0.001, 0.1,.01,0.001, 0.1,.01,0.001], 
               "R2":[sig1, sig2, sig3, relu1, relu2, relu3, elu1, elu2, elu3],
               "R2 CV":[sig_cv1, sig_cv2, sig_cv3, relu_cv1, relu_cv2, relu_cv3, elu_cv1, elu_cv2, elu_cv3],
               "R2 Adjusted":[sig_bar1,sig_bar2,sig_bar3,relu_bar1,relu_bar2,relu_bar3,elu_bar1,elu_bar2,elu_bar3]},
               index = ["sigm", "sigm", "sigm", "relu","relu","relu","elu", "elu", "elu"])
R2s

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

0epoch [00:00, ?epoch/s]

...

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

,Learning Rate,R2,R2 CV,R2 Adjusted
sigm,0.001,0.452911,0.456469,0.452129
sigm,0.010,0.457368,0.459942,0.456593
sigm,0.100,0.411292,0.420540,0.410450
relu,0.001,0.459215,0.450429,0.458442
relu,0.010,0.461984,0.463735,0.461215
relu,0.100,0.448571,0.449919,0.447782
elu,0.001,0.457950,0.455129,0.457175
elu,0.010,0.460305,0.461319,0.459534
elu,0.100,0.449200,0.449487,0.448412


# Forward 2 Layer Model

In [11]:
#Selecting features to feed the NN with forward selection and splitting training and testing sets (80/20)
reg = LinearRegression() 
sfs = SequentialFeatureSelector(estimator=reg)
sfs.fit(X,y)
forward = sfs.transform(X)
x_train, y_train = forward[-7006:], (y[-7006:]).to_numpy()
x_test, y_test   = forward[:-7006], (y[:-7006]).to_numpy()

#Calling Neural Net Function for 3 different activation functions and 3 different learning rates.
sig1, sig_cv1, sig_bar1 = TwoL_NN(5, 'sigmoid', 200, 0.1)
sig2, sig_cv2, sig_bar2 = TwoL_NN(5, 'sigmoid', 200, 0.01)
sig3, sig_cv3, sig_bar3 = TwoL_NN(5, 'sigmoid', 500, 0.001)

relu1, relu_cv1, relu_bar1 = TwoL_NN(5, 'relu', 200, 0.1)
relu2, relu_cv2, relu_bar2 = TwoL_NN(5, 'relu', 200, 0.01)
relu3, relu_cv3, relu_bar3 = TwoL_NN(5, 'relu', 500, 0.001)

elu1, elu_cv1, elu_bar1 = TwoL_NN(5, 'elu', 200, 0.1)
elu2, elu_cv2, elu_bar2 = TwoL_NN(5, 'elu', 200, 0.01)
elu3, elu_cv3, elu_bar3 = TwoL_NN(5, 'elu', 500, 0.001)

#Output
forward_R2s = pd.DataFrame({"Learning Rate" :[0.1,.01,0.001, 0.1,.01,0.001, 0.1,.01,0.001], 
               "R2":[sig1, sig2, sig3, relu1, relu2, relu3, elu1, elu2, elu3],
               "R2 CV":[sig_cv1, sig_cv2, sig_cv3, relu_cv1, relu_cv2, relu_cv3, elu_cv1, elu_cv2, elu_cv3],
               "R2 Adjusted":[sig_bar1,sig_bar2,sig_bar3,relu_bar1,relu_bar2,relu_bar3,elu_bar1,elu_bar2,elu_bar3]},
               index = ["sigm", "sigm", "sigm", "relu","relu","relu","elu", "elu", "elu"])

forward_R2s

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

0epoch [00:00, ?epoch/s]

...

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

,Learning Rate,R2,R2 CV,R2 Adjusted
sigm,0.001,0.455398,0.453744,0.455087
sigm,0.010,0.453013,0.454573,0.452701
sigm,0.100,0.439482,0.446831,0.439162
relu,0.001,0.466396,0.458654,0.466091
relu,0.010,0.457598,0.454473,0.457288
relu,0.100,0.442403,0.447147,0.442085
elu,0.001,0.463578,0.458433,0.463272
elu,0.010,0.460512,0.454121,0.460203
elu,0.100,0.447832,0.446999,0.447517
